In [3]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import cv2

In [4]:
path1=r"C:\mask_and_nomask"
cate=['with_mask','without_mask']

In [5]:
input_image=[]
image_size=200
for i in cate:
    folders=os.path.join(path1,i)
    label=cate.index(i) # to tell software which image is cat and dog
    for image in os.listdir(folders):
        image_path=os.path.join(folders,image)
        image_array=cv2.imread(image_path) # use cv2 to read and store image
        image_array=cv2.resize(image_array,(image_size,image_size))
        input_image.append([image_array,label])




In [6]:
len(input_image)    #mask + no mask

7553

In [22]:
# np.random.shuffle(input_image) 
# add here code for shuffling
# if you do shuffling then dont do train test sampling 

# x_train = X[0:7000]
# x_test = X[7000:8833]

# y_train = Y[0:7000]
# y_test = Y[7000:8833]

In [24]:
X=[]
Y=[]
# separating x i.e pixels and labels will extract y separate
for X_values,labels in input_image:
    X.append(X_values)
    Y.append(labels)

In [25]:
X=np.array(X)
Y=np.array(Y)

In [9]:
X=X/255

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [11]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout


In [12]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', input_shape = X.shape[1:]))

model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(x_train,y_train,validation_split=0.2,batch_size=64,epochs=10)

Epoch 1/10


76/76 [==============================] - 51s 608ms/step - loss: 2.0065 - accuracy: 0.7445 - val_loss: 0.3831 - val_accuracy: 0.8503
Epoch 2/10
76/76 [==============================] - 39s 509ms/step - loss: 0.2494 - accuracy: 0.9040 - val_loss: 0.2739 - val_accuracy: 0.9049
Epoch 3/10
76/76 [==============================] - 40s 525ms/step - loss: 0.1644 - accuracy: 0.9408 - val_loss: 0.2600 - val_accuracy: 0.9090
Epoch 4/10
76/76 [==============================] - 41s 534ms/step - loss: 0.1262 - accuracy: 0.9549 - val_loss: 0.3340 - val_accuracy: 0.8933
Epoch 5/10
76/76 [==============================] - 39s 510ms/step - loss: 0.0581 - accuracy: 0.9822 - val_loss: 0.3493 - val_accuracy: 0.9041
Epoch 6/10
76/76 [==============================] - 40s 528ms/step - loss: 0.0332 - accuracy: 0.9913 - val_loss: 0.4216 - val_accuracy: 0.8850
Epoch 7/10
76/76 [==============================] - 39s 518ms/step - loss: 0.0338 - accuracy: 0.9911 - val_loss: 0.4256 - val_accuracy: 0.89

In [14]:
import pandas as pd

In [15]:
df=pd.DataFrame(model.history.history)
df# it's like a generator that runs one time and then disappears

,loss,accuracy,val_loss,val_accuracy
0,2.006462,0.744465,0.383071,0.850290
1,0.249385,0.903993,0.273873,0.904880
2,0.164389,0.940823,0.260007,0.909016
3,0.126193,0.954893,0.334008,0.893300
4,0.058115,0.982206,0.349311,0.904053
5,0.033218,0.991310,0.421566,0.885029
6,0.033754,0.991103,0.425641,0.896609
7,0.058187,0.986965,0.452750,0.895782
8,0.043016,0.987999,0.393559,0.904880
9,0.015170,0.996689,0.500564,0.902399


In [16]:
pred=model.predict(x_test)
pred

48/48 [==============================] - 4s 58ms/step


array([[9.9477345e-01, 5.2266279e-03],
       [9.9999976e-01, 1.8821413e-07],
       [2.3809425e-03, 9.9761909e-01],
       ...,
       [3.8123133e-05, 9.9996185e-01],
       [9.8348355e-01, 1.6516453e-02],
       [1.0000000e+00, 9.1247987e-09]], dtype=float32)

In [17]:
pred_classes=pred.argmax(axis=1)

In [18]:
from sklearn.metrics import confusion_matrix,classification_report

In [19]:
cm=confusion_matrix(y_test,pred_classes)
cm

array([[682,  74],
       [ 90, 665]], dtype=int64)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_classes))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       756
           1       0.90      0.88      0.89       755

    accuracy                           0.89      1511
   macro avg       0.89      0.89      0.89      1511
weighted avg       0.89      0.89      0.89      1511

